In [1]:
import sys
from data import InputData, OutputData, Config, WorkShop
from typing import List

In [2]:
def Max(a, b):
    if a > b:
        return a
    else:
        return b
# Topological sort
class Queue:
    def __init__(self):
        self.vec = []
        self.headIndex = 0
        self.tailIndex = 0
        return

    def Push(self, item):
        self.vec.append(item)
        self.tailIndex = self.tailIndex + 1
        return

    def IsEmpty(self):
        return self.headIndex == self.tailIndex

    def Pop(self):
        if self.headIndex >= self.tailIndex:
            print("[Err] Queue Pop")
            return "Err"
        item = self.vec[self.headIndex]
        self.headIndex = self.headIndex + 1
        return item


In [3]:
inputData = InputData.from_file('../sample/sample.in')

In [4]:
workshops: List[WorkShop] = []
for id in range(inputData.N):
    workshops.append(WorkShop())


In [5]:
workshops[0]

In [7]:
# Count the earliest and latest time that one workshop can enter
for wid in range(inputData.M):
    window = inputData.windows[wid]
    workshop = workshops[window.workshopIndex]
    minTi = wid
    maxTi = inputData.L * inputData.M + wid
    print(wid, minTi, maxTi)
    if minTi < workshop.minTi:
        workshop.minTi = minTi
    if maxTi > workshop.maxTi:
        workshop.maxTi = maxTi


0 0 2
1 1 3


Probe workshop

In [8]:
for workshop in workshops:
  print(workshop)

minTi:0	maxTi3	anyRidOfEngine[4294967295, 4294967295, 4294967295]

minTi:4294967295	maxTi0	anyRidOfEngine[4294967295, 4294967295, 4294967295]

minTi:4294967295	maxTi0	anyRidOfEngine[4294967295, 4294967295, 4294967295]

minTi:4294967295	maxTi0	anyRidOfEngine[4294967295, 4294967295, 4294967295]



In [8]:
type(workshops[0])

data.WorkShop

In [9]:
widOfTi = []
for loopIndex in range(inputData.L + 1):
    for wid in range(inputData.M):
        widOfTi.append(wid)


In [10]:
for wid in range(inputData.M, inputData.W):
    window = inputData.windows[wid]
    workshop = workshops[window.workshopIndex]
    widOfTi.append(wid)
    minTi = len(widOfTi) - 1
    if minTi < workshop.minTi:
        workshop.minTi = minTi
    if window.canSelfLoop:
        for loopIndex in range(inputData.L):
            widOfTi.append(wid)
    maxTi = len(widOfTi) - 1
    if maxTi > workshop.maxTi:
        workshop.maxTi = maxTi


In [11]:
workshops.sort(key=lambda de: de.minTi)


In [12]:
isDeviceInPipeline = [False] * inputData.D
for eid in inputData.pipeline.edgeIndexs:
    edge = inputData.edges[eid]
    isDeviceInPipeline[edge.sendDevice] = True
    isDeviceInPipeline[edge.recvDevice] = True


In [13]:
# Make statistics of the workshop area where
# Collect statistics on the workshop region that support a certain
# type of equipment in the workshop.
for rid in range(inputData.R):
    region = inputData.regions[rid]
    nid = region.workshopIndex
    workshop = workshops[nid]
    if region.energyType == 0:
        workshop.anyRidOfEngine[0] = rid
        workshop.anyRidOfEngine[1] = rid
    elif region.energyType == 1:
        workshop.anyRidOfEngine[0] = rid
    elif region.energyType == 2:
        workshop.anyRidOfEngine[1] = rid
    elif region.energyType == 3:
        workshop.anyRidOfEngine[2] = rid
    elif region.energyType == 4:
        workshop.anyRidOfEngine[2] = rid


In [14]:
nextEdgeMgr = []
prevEdgeMgr = []
for did in range(inputData.D):
    nextEdgeMgr.append([])
    prevEdgeMgr.append([])

for eid in range(inputData.E):
    edge = inputData.edges[eid]
    nextEdgeMgr[edge.sendDevice].append(eid)
    prevEdgeMgr[edge.recvDevice].append(eid)


In [15]:
queue = Queue()
inCnt = [0] * inputData.D
for did in range(inputData.D):
    inCnt[did] = len(prevEdgeMgr[did])
    if inCnt[did] == 0:
        queue.Push(did)

ridOfDid = [Config.MAX_U32] * inputData.D
minTiOfDid = [0] * inputData.D

pid = 0
preTi = 0
widOfPid = [Config.MAX_U32] * (inputData.pipeline.edgeNum + 1)

while not queue.IsEmpty():
    curDid = queue.Pop()
    if isDeviceInPipeline[curDid]:
        startTi = minTiOfDid[curDid]
        engineType = inputData.devices[curDid].engineType
        if pid != 0:
            edge = inputData.edges[inputData.pipeline.edgeIndexs[pid - 1]]
            startTi = max(startTi, preTi + (edge.type == 0))

        for ti in range(startTi, len(widOfTi)):
            wid = widOfTi[ti]
            window = inputData.windows[wid]
            if not window.enginesSupport[engineType]:
                continue
            workshop = workshops[window.workshopIndex]
            rid = workshop.anyRidOfEngine[engineType]
            if rid == Config.MAX_U32:
                continue

            ridOfDid[curDid] = rid
            widOfPid[pid] = wid
            pid = pid + 1
            preTi = ti
            break
    else:
        engineType = inputData.devices[curDid].engineType
        for i in range(inputData.N):
            workshop = workshops[i]
            rid = workshop.anyRidOfEngine[engineType]
            if rid == Config.MAX_U32:
                continue
            if workshop.maxTi >= minTiOfDid[curDid]:
                ridOfDid[curDid] = rid
                break
    if ridOfDid[curDid] == Config.MAX_U32:
        print("wrong in %d" % curDid)
        exit()
    workshop = workshops[inputData.regions[ridOfDid[curDid]].workshopIndex]
    for eid in nextEdgeMgr[curDid]:
        edge = inputData.edges[eid]
        curDid = edge.recvDevice
        if edge.type == 0:
            requestTi = workshop.minTi + 1
        else:
            requestTi = workshop.minTi
        minTiOfDid[curDid] = Max(minTiOfDid[curDid], requestTi)
        inCnt[curDid] = inCnt[curDid] - 1
        if inCnt[curDid] == 0:
            queue.Push(curDid)


In [16]:
outputData = OutputData(
    deviceNum=inputData.D,
    regionIndexs=ridOfDid,
    stepNum=inputData.pipeline.edgeNum + 1,
    timeWindowIndexs=widOfPid,
)


In [17]:

outputData.print()


13
1 2 1 4 1 5 1 1 4 7 1 5 5
7
0 1 0 0 1 3 4
